## RAG pipeline 
Команда Утики MISIS

In [ ]:
import torch
import numpy as np
import pandas as pd
import re
import torch
import nltk
import joblib
import pandas as pd
from IPython.display import display, Markdown

from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
from transformers import set_seed
from sentence_transformers import SentenceTransformer

from sklearn.neighbors import NearestNeighbors

from utils.utils import Chunker, Generator
from utils.spell_checker import SpellChecker
from utils.llm_config import GenerationConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

### Загружаем модель ретривера и модель генератора

In [ ]:
retriever_model = SentenceTransformer("BAAI/bge-m3", device='cuda')
retriever_model.max_seq_length = 512

generator_model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).cuda().eval()

### Загружаем данные, тут формат работы в ноутбуке, этот пункт будет заменен на обращение к ClickHouse

In [3]:
texts = pd.read_csv('data/documents.csv', header=None, names=['url', 'text'])
texts = texts.dropna()
texts['text'] = texts['text'].apply(lambda x: re.sub(' +', ' ', x))
texts['text'] = texts['text'].apply(lambda x: re.sub('\n+', '\n', x))
texts.url = texts.url.apply(lambda x: x if  x[-1] == '/' else x+'/')
texts.text.apply(lambda x: x.replace(x[x.find('<!--'): x.rfind('-->')+3], '').replace('Complex', '').strip('\n '))
texts = texts.set_index('url')

#### Инициализируем функцию, которая будет бить наши документы на чанки. Инициализируем генератор.

In [4]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)
generator = Generator(generator_tokenizer, generator_model, config=GenerationConfig)

In [5]:
chunks, urls = chunker.split_texts(texts)
chunks, urls = np.array(chunks), np.array(urls)

len(chunks)

35230

#### Либо грузим готовые эмбединги, либо делаем их сами

In [6]:
# embeddings = model.encode(chunks, batch_size=32, normalize_embeddings=True, show_progress_bar=True)
# joblib.dump(embeddings, 'bge-m3-new-clear.pkl')

embeddings = joblib.load('bge-m3-new-clear.pkl')
embeddings.shape

(35230, 1024)

#### Инициализируем KNN 

In [7]:
knn = NearestNeighbors(metric='cosine')
knn.fit(embeddings)

NearestNeighbors(metric='cosine')

### Загружаем бенчмарк вопросы

In [2]:
queries = pd.read_csv('data/benchmarks.csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

In [3]:
question = queries.question[6]
question

'Что такое ВЕБ.РФ?'

### Тестируем

In [10]:
def retrieve(question, model, knn, chunks, urls):
    skores, neighbors = knn.kneighbors(model.encode(question, batch_size=32, normalize_embeddings=True).reshape(1, -1), 3)
    neighbors = neighbors.squeeze()
    retrieved_texts = list(chunks[neighbors])
    retrieved_urls = list(set(urls[neighbors]))
    return retrieved_texts, retrieved_urls

In [11]:
retrieved_texts, retrieved_urls = retrieve(question, retriever_model, knn, chunks, urls)

In [12]:
display(Markdown(generator.get_answer(question, retrieved_texts, retrieved_urls, temperature=0.8)))

Государственная корпорация развития "ВЭБ.РФ".

Использованные документы:
1) http://pravo.gov.ru/proxy/ips/?docbody=&nd=102114195/ 


In [14]:
%timeit generator.get_answer(question, retrieved_texts, retrieved_urls, temperature=0.8)

1.08 s ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Дополнительно

#### HyDE

In [15]:
generator.hyde(question, temperature=1, num_beams=2)

'Что такое ВЕБ.РФ?'

#### Spell Checker

In [6]:
SPELL_CHECKER_MODEL_NAME = 'UrukHan/t5-russian-spell'

spell_checker_tokenizer = T5TokenizerFast.from_pretrained(SPELL_CHECKER_MODEL_NAME)
spell_checker_model = AutoModelForSeq2SeqLM.from_pretrained(SPELL_CHECKER_MODEL_NAME).to(device)

In [8]:
spell_checker = SpellChecker(spell_checker_tokenizer, spell_checker_model)

In [12]:
spell_checker.get_answer('а чт таке конституц росииск федерации')

['А что такое Конституция Российской Федерации?']